# Get Data

In [112]:
import os
from pathlib import Path
import pandas as pd
import datetime
from datetime import datetime

In [113]:

# NOTE: This wont work for jupyter so we are using sth else for now
# # Get the absolute path of the current script (jupyter dir)
# BASE_DIR = Path(__file__).resolve().parent

# # Build the path to the data directory
# DATA_DIR = BASE_DIR / 'data'
BASE_DIR = Path.cwd()

# Build the path to the data directory
DATA_DIR = BASE_DIR.parents[0] / 'transfer_data' 

# # Example of using the path to the data directory
# csv_file = DATA_DIR / 'your_file.csv'
# print(csv_file)

In [114]:
# import all files in Data folder and read into dataframes
dataframes=[]


# Below is just to surpress warnings...
competitions_df = pd.DataFrame
appearances_df = pd.DataFrame
player_valuations_df = pd.DataFrame
game_events_df = pd.DataFrame
players_df= pd.DataFrame
games_df = pd.DataFrame
club_games_df = pd.DataFrame
clubs_df = pd.DataFrame

# Actual reading csv flies
for dirpath, dirname, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        file=filename.split('.')
        file=((file[0]+"_df"))
        if file !="_df":
            filepath=os.path.join(dirpath,filename)
            df=pd.read_csv(filepath,sep=",",encoding = "UTF-8")
            exec(f'{file} = df.copy()')
            print(file, df.shape)
            dataframes.append(df)
print('Data imported')



competitions_df (43, 10)
appearances_df (1166215, 13)
player_valuations_df (424323, 9)
game_events_df (559962, 7)
players_df (28459, 23)
games_df (61538, 21)
club_games_df (123076, 11)
clubs_df (411, 16)
Data imported


# Process data

## Players_df

In [115]:
players_df.shape

(28459, 23)

In [116]:
# Calculate the age of each player (object -> datetime)
players_df['date_of_birth'] = pd.to_datetime(players_df['date_of_birth']) 
# drop players with no date of birth
players_df.dropna(subset = ['date_of_birth'], inplace=True)
players_df.shape

(28417, 23)

In [117]:
# Get age column of players
now = datetime.now()
players_df['age'] = (now - players_df['date_of_birth']).apply(lambda x: x.days / 365.25)
players_df['age'] = players_df['age'].round().astype(int)
# players_df['age'] = players_df['age'].round().astype(int) 


In [118]:
elo_df = players_df.copy()


# Initialisation
## Getting club elo
Using clubelo.com's API, get club elo

In [119]:
import io
import requests

# Get a date and data
response = requests.get(f"http://api.clubelo.com/{datetime.today().strftime('%Y-%m-%d')}")
club_elo_df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
club_elo_df.drop('Rank', axis=1)

,Club,Country,Level,Elo,From,To
0,Man City,ENG,1,2060.798584,2024-09-20,2024-09-22
1,Real Madrid,ESP,1,1995.374390,2024-09-20,2024-09-21
2,Inter,ITA,1,1963.372559,2024-09-20,2024-09-22
3,Arsenal,ENG,1,1959.767212,2024-09-20,2024-09-22
4,Leverkusen,GER,1,1939.445068,2024-09-20,2024-09-22
...,...,...,...,...,...,...
634,FCB Magpies,GIB,0,896.816284,2024-08-30,2024-10-03
635,Atletic Club Escaldes,AND,0,880.255310,2024-08-30,2024-10-03
636,SS Virtus,SMR,0,708.203552,2024-08-02,2024-10-03
637,Tre Penne,SMR,0,695.522400,2024-08-02,2024-10-03


## Now, initialise player
- Find out the how far our data covers

In [120]:
games_df.shape

(61538, 21)

In [121]:
# Calculate the age of each player (object -> datetime)
games_df['date'] = pd.to_datetime(games_df['date']) 
# drop players with no date of birth
# games_df.dropna(subset = ['date'], inplace=True)
games_df.shape

(61538, 21)

In [122]:
games_df.sort_values(by=['date'])
games_df.tail()

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,home_club_manager_name,away_club_manager_name,stadium,attendance,referee,url,home_club_name,away_club_name,aggregate,competition_type
61533,2888300,NL1,2017,26. Matchday,2018-03-03,383,200,3,0,1,...,Phillip Cocu,Jean-Paul de Jong,Philips Stadion,33800.0,Kevin Blom,https://www.transfermarkt.co.uk/psv-eindhoven_...,PSV Eindhoven,FC Utrecht,3:0,domestic_league
61534,3450596,NL1,2020,12. Matchday,2020-12-13,383,200,2,1,2,...,Roger Schmidt,René Hake,Philips Stadion,NaN,Serdar Gözübüyük,https://www.transfermarkt.co.uk/psv-eindhoven_...,PSV Eindhoven,FC Utrecht,2:1,domestic_league
61535,3597835,NL1,2021,6. Matchday,2021-09-22,467,200,0,3,10,...,Rogier Meijer,René Hake,Goffertstadion,8333.0,Marc Nagtegaal,https://www.transfermarkt.co.uk/nec-nijmegen_f...,NEC Nijmegen,FC Utrecht,0:3,domestic_league
61536,2250211,NL1,2012,5. Matchday,2012-09-16,200,383,1,0,8,...,Jan Wouters,Dick Advocaat,"Stadion ""Galgenwaard""",17243.0,Kevin Blom,https://www.transfermarkt.co.uk/fc-utrecht_psv...,FC Utrecht,PSV Eindhoven,1:0,domestic_league
61537,4031353,UKR1,2022,17. Matchday,2023-03-12,48726,48332,0,1,12,...,Leonid Kuchuk,Yaroslav Vyshnyak,Arena Lviv,NaN,Oleksandr Omelchenko,https://www.transfermarkt.co.uk/rukh-lviv_kolo...,Rukh Lviv,Kolos Kovalivka,0:1,domestic_league


In [110]:
# games_df.sort_values(by=['date'])
oldest_date = games_df['date'].min()
newest_date = games_df['date'].max()

In [60]:

# # Calculate the contract remaining of each player
# players_df['contract_expiration_date'] = pd.to_datetime(players_df['contract_expiration_date'])
# # drop players with no date of birth
# players_df = players_df[players_df['contract_expiration_date'].isnull() == False]
# now = datetime.now()
# players_df['term_days_remaining'] = (players_df['contract_expiration_date']- now).apply(lambda x: x.days) 


/var/folders/nh/kbd4vyw93l9_cfg59h74rpwh0000gn/T/ipykernel_4550/1132561174.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_df['term_days_remaining'] = (players_df['contract_expiration_date']- now).apply(lambda x: x.days)


ValueError: unconverted data remains when parsing with format "%Y-%m-%d": " 00:00:00", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [59]:
players_df

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,height_in_cm,market_value_in_eur,highest_market_value_in_eur,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,age
0,597,Aleksandr,Hleb,Aleksandr Hleb,2016,2696,aleksandr-hleb,UdSSR,Minsk,Belarus,...,NaN,NaN,19000000.0,NaN,fair-sport GmbH,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aleksandr-hleb...,RU1,Krylya Sovetov Samara,43
1,1428,Mike,Hanke,Mike Hanke,2013,60,mike-hanke,Germany,Hamm,Germany,...,185.0,NaN,5000000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/mike-hanke/pro...,L1,SC Freiburg,41
2,1560,Youssef,El Akchaoui,Youssef El Akchaoui,2012,306,youssef-el-akchaoui,Netherlands,Dordrecht,Morocco,...,NaN,NaN,1750000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/youssef-el-akc...,NL1,SC Heerenveen,44
4,1726,Heiko,Westermann,Heiko Westermann,2016,610,heiko-westermann,Germany,Alzenau,Germany,...,190.0,NaN,9000000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/heiko-westerma...,NL1,Ajax Amsterdam,41
5,2306,Markus,Miller,Markus Miller,2014,42,markus-miller,Germany,Lindenberg im Allgäu,Germany,...,189.0,NaN,3500000.0,NaN,ARP Sportmarketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/markus-miller/...,L1,NaN,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28454,199337,Rifat,Erdem,Rifat Erdem,2013,2293,rifat-erdem,Turkey,Konya,Turkey,...,NaN,NaN,100000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/rifat-erdem/pr...,TR1,Konyaspor,35
28455,209569,Mücahit,Atalay,Mücahit Atalay,2020,2293,mucahit-atalay,Turkey,Trabzon,Turkey,...,186.0,NaN,250000.0,2023-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/mucahit-atalay...,TR1,Konyaspor,33
28456,221118,Patrick Friday,Eze,Patrick Friday Eze,2017,2293,patrick-friday-eze,Nigeria,Kaduna,Nigeria,...,187.0,200000.0,775000.0,2023-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/patrick-friday...,TR1,Konyaspor,32
28457,273053,Burak,Saban,Burak Saban,2018,2293,burak-saban,Netherlands,Hoorn,Netherlands,...,177.0,NaN,200000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/burak-saban/pr...,TR1,Konyaspor,30


In [ ]:
# drop players with no date of birth
# players_df = players_df[players_df['date_of_birth'].isnull() == False]
# now = datetime.now()
# players_df['age'] = (now - players_df['date_of_birth']).apply(lambda x: x.days) / 365.25
# players_df['age'] = players_df['age'].round().astype(int) 

# # Calculate the contract remaining of each player
# players_df['contract_expiration_date'] = pd.to_datetime(players_df['contract_expiration_date'])
# # drop players with no date of birth
# players_df = players_df[players_df['contract_expiration_date'].isnull() == False]
# now = datetime.now()
# players_df['term_days_remaining'] = (players_df['contract_expiration_date']- now).apply(lambda x: x.days) 

# # add year to player valuations
# player_valuations_df['datetime']=pd.to_datetime(player_valuations_df['datetime'], format="%Y-%m-%d")
# player_valuations_df['year']=player_valuations_df['datetime'].dt.year

# # add year to player appearances
# appearances_df['datetime']=pd.to_datetime(appearances_df['date'], format="%Y-%m-%d")
# appearances_df['year']=appearances_df['datetime'].dt.year

# #add position to player valuations
# position_df=players_df.copy()
# position_df=position_df.drop(['name', 'current_club_id', 'current_club_name', 'country_of_citizenship', 'country_of_birth', 'city_of_birth', 'date_of_birth', 'foot', 'height_in_cm', 'market_value_in_eur', 'highest_market_value_in_eur', 'agent_name', 'contract_expiration_date', 'current_club_domestic_competition_id', 'first_name', 'last_name', 'player_code', 'image_url', 'last_season', 'url', 'age'], axis=1)
# player_valuations_df=player_valuations_df.merge(position_df, left_on='player_id', right_on='player_id')

# #add position to appearances
# value_df=players_df.copy()
# value_df=value_df.drop(['name', 'current_club_id', 'current_club_name', 'country_of_citizenship', 'country_of_birth', 'city_of_birth', 'date_of_birth', 'foot', 'height_in_cm', 'sub_position', 'highest_market_value_in_eur', 'agent_name', 'contract_expiration_date', 'current_club_domestic_competition_id', 'first_name', 'last_name', 'player_code', 'image_url', 'last_season', 'url', 'age'], axis=1)
# appearances_df1=value_df.merge(appearances_df, left_on='player_id', right_on='player_id')

# print("Data processing complete")